# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921735467447                   -0.69    6.0    284ms
  2   -7.926165030181       -2.35       -1.22    1.0    158ms
  3   -7.926839200877       -3.17       -2.37    1.9    202ms
  4   -7.926861195779       -4.66       -3.03    2.1    189ms
  5   -7.926861646855       -6.35       -3.38    2.0    180ms
  6   -7.926861668295       -7.67       -3.75    1.5    178ms
  7   -7.926861679158       -7.96       -4.13    1.1    153ms
  8   -7.926861681799       -8.58       -5.11    1.4    158ms
  9   -7.926861681864      -10.19       -5.26    3.1    195ms
 10   -7.926861681872      -11.09       -6.43    1.0    153ms
 11   -7.926861681873      -12.28       -6.67    3.0    202ms
 12   -7.926861681873   +    -Inf       -7.26    1.0    185ms
 13   -7.926861681873   +    -Inf       -7.85    2.0    166ms
 14   -7.926861681873   +    -Inf       -8.09    2.0    179ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921718858140                   -0.69    6.1    227ms
  2   -7.926162098076       -2.35       -1.22    1.0    180ms
  3   -7.926838830956       -3.17       -2.37    1.9    179ms
  4   -7.926861198054       -4.65       -3.03    2.2    232ms
  5   -7.926861646557       -6.35       -3.38    2.1    174ms
  6   -7.926861668205       -7.66       -3.75    1.6    157ms
  7   -7.926861679057       -7.96       -4.12    1.1    150ms
  8   -7.926861681798       -8.56       -5.10    1.6    159ms
  9   -7.926861681862      -10.19       -5.23    3.0    194ms
 10   -7.926861681872      -11.02       -6.34    1.0    170ms
 11   -7.926861681873      -12.29       -6.62    2.9    201ms
 12   -7.926861681873      -14.05       -7.13    1.0    152ms
 13   -7.926861681873   +    -Inf       -7.77    1.5    157ms
 14   -7.926861681873      -15.05       -7.97    2.2    183ms
 15   -7.

The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921715832079                   -0.69    6.0    309ms
  2   -7.926167218622       -2.35       -1.22    1.0    163ms
  3   -7.926842014899       -3.17       -2.37    1.9    187ms
  4   -7.926864605795       -4.65       -3.02    2.2    223ms
  5   -7.926865050269       -6.35       -3.34    1.8    175ms
  6   -7.926865075632       -7.60       -3.69    1.5    158ms
  7   -7.926865090446       -7.83       -4.15    1.1    185ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
